# Data Cleaning and Multi-Timeframe Aggregation

This notebook processes the raw 1-minute BTCUSD dataset by:
1. Cleaning the data (removing initial rows with no feature data)
2. Imputing remaining NaN values
3. Aggregating to multiple timeframes (15min, 30min, 1day)
4. Properly handling pre-calculated features during aggregation

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

## 1. Load and Examine Raw Dataset

In [4]:
# Load the raw dataset
raw_data_path = 'E:\Coding\Crypto_Research\data\BTCUSD_2023-1min_ML_with_FDs.csv'
df = pd.read_csv(raw_data_path)

print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

Dataset shape: (484640, 28)
Columns: ['Timestamp', 'Unnamed: 0.1', 'Unnamed: 0', 'Open', 'High', 'Low', 'Close', 'Volume', 'ema_7d', 'ema_20d', 'ema_30d', 'sma_7d', 'sma_20d', 'sma_30d', 'macd_12_26', 'macd_sig_12_26', 'macd_hist_12_26', 'rsi_14d', 'bb_mid_20d', 'bb_upper_20d', 'bb_lower_20d', 'atr_14d', 'fd_14d', 'future_close', 'future_return', 'future_trend', 'fd_7d', 'fd_30d']

First few rows:


,Timestamp,Unnamed: 0.1,Unnamed: 0,Open,High,Low,Close,Volume,ema_7d,ema_20d,...,bb_mid_20d,bb_upper_20d,bb_lower_20d,atr_14d,fd_14d,future_close,future_return,future_trend,fd_7d,fd_30d
0,2023-01-01 14:00:00,0,0,16474.3,16475.3,16473.1,16475.3,0.000088,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,17180.3,0.042791,1,NaN,NaN
1,2023-01-01 14:01:00,1,1,16475.1,16477.7,16474.7,16476.8,0.000086,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,17182.3,0.042818,1,NaN,NaN
2,2023-01-01 14:02:00,2,2,16477.3,16478.2,16476.4,16477.1,0.000086,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,17183.2,0.042853,1,NaN,NaN
3,2023-01-01 14:03:00,3,3,16476.9,16478.5,16476.1,16478.2,0.000092,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,17181.4,0.042675,1,NaN,NaN
4,2023-01-01 14:04:00,4,4,16477.6,16478.9,16477.0,16478.2,0.000070,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,17181.4,0.042675,1,NaN,NaN


In [5]:
# Check data types and missing values
print("Data types:")
print(df.dtypes)
print(f"\nMissing values per column:")
missing_counts = df.isnull().sum()
print(missing_counts[missing_counts > 0])

Data types:
Timestamp           object
Unnamed: 0.1         int64
Unnamed: 0           int64
Open               float64
High               float64
Low                float64
Close              float64
Volume             float64
ema_7d             float64
ema_20d            float64
ema_30d            float64
sma_7d             float64
sma_20d            float64
sma_30d            float64
macd_12_26         float64
macd_sig_12_26     float64
macd_hist_12_26    float64
rsi_14d            float64
bb_mid_20d         float64
bb_upper_20d       float64
bb_lower_20d       float64
atr_14d            float64
fd_14d             float64
future_close       float64
future_return      float64
future_trend         int64
fd_7d              float64
fd_30d             float64
dtype: object

Missing values per column:
ema_7d             10079
ema_20d            28799
ema_30d            43199
sma_7d             10079
sma_20d            28799
sma_30d            43199
macd_12_26         37439
macd_sig_12_26 

In [6]:
# Convert timestamp column to datetime if needed
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
elif 'time' in df.columns:
    df['timestamp'] = pd.to_datetime(df['time'])
    df = df.drop('time', axis=1)
elif 'datetime' in df.columns:
    df['timestamp'] = pd.to_datetime(df['datetime'])
    df = df.drop('datetime', axis=1)
else:
    # Assume first column is timestamp
    timestamp_col = df.columns[0]
    df['timestamp'] = pd.to_datetime(df[timestamp_col])
    if timestamp_col != 'timestamp':
        df = df.drop(timestamp_col, axis=1)

# Set timestamp as index
df = df.set_index('timestamp')
df = df.sort_index()

print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Total time span: {df.index.max() - df.index.min()}")

Date range: 2023-01-01 14:00:00 to 2023-12-31 14:27:00
Total time span: 364 days 00:27:00


## 2. Data Cleaning

In [7]:
# Identify OHLCV columns
ohlcv_cols = []
for col in ['open', 'high', 'low', 'close', 'volume']:
    if col in df.columns:
        ohlcv_cols.append(col)
    elif col.upper() in df.columns:
        ohlcv_cols.append(col.upper())
    elif col.capitalize() in df.columns:
        ohlcv_cols.append(col.capitalize())

print(f"OHLCV columns identified: {ohlcv_cols}")

# Identify feature columns (everything except OHLCV)
feature_cols = [col for col in df.columns if col not in ohlcv_cols]
print(f"Feature columns count: {len(feature_cols)}")
print(f"First 10 feature columns: {feature_cols[:10]}")

OHLCV columns identified: ['Open', 'High', 'Low', 'Close', 'Volume']
Feature columns count: 22
First 10 feature columns: ['Unnamed: 0.1', 'Unnamed: 0', 'ema_7d', 'ema_20d', 'ema_30d', 'sma_7d', 'sma_20d', 'sma_30d', 'macd_12_26', 'macd_sig_12_26']


In [8]:
# Find the first day where features have data
first_valid_feature_idx = df[feature_cols].dropna().index[0] if len(df[feature_cols].dropna()) > 0 else df.index[0]
first_day = first_valid_feature_idx.date()

print(f"First valid feature data starts at: {first_valid_feature_idx}")
print(f"Removing data before: {first_day}")

# Remove data before first valid feature day
df_cleaned = df[df.index.date >= first_day].copy()

print(f"Original dataset shape: {df.shape}")
print(f"Cleaned dataset shape: {df_cleaned.shape}")
print(f"Removed {df.shape[0] - df_cleaned.shape[0]} rows")

First valid feature data starts at: 2023-02-10 03:16:00
Removing data before: 2023-02-10
Original dataset shape: (484640, 27)
Cleaned dataset shape: (434438, 27)
Removed 50202 rows


## 3. Impute Remaining NaN Values

In [9]:
# Check remaining missing values
missing_after_cleaning = df_cleaned.isnull().sum()
print("Missing values after cleaning:")
print(missing_after_cleaning[missing_after_cleaning > 0])

# Impute missing values
df_imputed = df_cleaned.copy()

# For OHLCV data, use forward fill then backward fill
for col in ohlcv_cols:
    if col in df_imputed.columns:
        df_imputed[col] = df_imputed[col].fillna(method='ffill').fillna(method='bfill')

# For feature columns, use forward fill then backward fill
for col in feature_cols:
    if col in df_imputed.columns:
        df_imputed[col] = df_imputed[col].fillna(method='ffill').fillna(method='bfill')

# Final check for any remaining NaN values
final_missing = df_imputed.isnull().sum()
if final_missing.sum() > 0:
    print("\nRemaining missing values after imputation:")
    print(final_missing[final_missing > 0])
    # Fill any remaining NaN with 0
    df_imputed = df_imputed.fillna(0)
else:
    print("\nNo missing values remaining after imputation")

Missing values after cleaning:
macd_sig_12_26      196
macd_hist_12_26     196
future_close       7200
future_return      7200
dtype: int64

No missing values remaining after imputation


## 4. Define Aggregation Functions

In [10]:
def create_aggregation_dict(df, ohlcv_cols, feature_cols):
    """
    Create aggregation dictionary for resampling.
    OHLCV gets standard OHLCV aggregation.
    Features get last value (since they're already calculated indicators).
    """
    agg_dict = {}
    
    # Standard OHLCV aggregation
    for col in ohlcv_cols:
        if col in df.columns:
            if col.lower() in ['open']:
                agg_dict[col] = 'first'
            elif col.lower() in ['high']:
                agg_dict[col] = 'max'
            elif col.lower() in ['low']:
                agg_dict[col] = 'min'
            elif col.lower() in ['close']:
                agg_dict[col] = 'last'
            elif col.lower() in ['volume']:
                agg_dict[col] = 'sum'
    
    # Features: use last value since they're pre-calculated indicators
    for col in feature_cols:
        if col in df.columns:
            agg_dict[col] = 'last'
    
    return agg_dict

def aggregate_timeframe(df, timeframe, ohlcv_cols, feature_cols):
    """
    Aggregate dataframe to specified timeframe.
    """
    agg_dict = create_aggregation_dict(df, ohlcv_cols, feature_cols)
    
    # Resample to target timeframe
    df_agg = df.resample(timeframe).agg(agg_dict)
    
    # Remove rows where OHLCV data is missing (market closed periods)
    if ohlcv_cols:
        df_agg = df_agg.dropna(subset=[col for col in ohlcv_cols if col in df_agg.columns])
    
    return df_agg

print("Aggregation functions defined")

Aggregation functions defined


## 5. Create Multi-Timeframe Datasets

In [11]:
# Create 15-minute dataset
print("Creating 15-minute dataset...")
df_15min = aggregate_timeframe(df_imputed, '15T', ohlcv_cols, feature_cols)
print(f"15-minute dataset shape: {df_15min.shape}")

Creating 15-minute dataset...
15-minute dataset shape: (30925, 27)


In [12]:
# Create 30-minute dataset
print("Creating 30-minute dataset...")
df_30min = aggregate_timeframe(df_imputed, '30T', ohlcv_cols, feature_cols)
print(f"30-minute dataset shape: {df_30min.shape}")

Creating 30-minute dataset...
30-minute dataset shape: (15466, 27)


In [16]:
# Create 1-hour dataset
print("Creating 1-hour dataset...")
df_1h = aggregate_timeframe(df_imputed, '1H', ohlcv_cols, feature_cols)
print(f"1-hour dataset shape: {df_1h.shape}")

Creating 1-hour dataset...
1-hour dataset shape: (7738, 27)


In [13]:
# Create daily dataset
print("Creating daily dataset...")
df_daily = aggregate_timeframe(df_imputed, '1D', ohlcv_cols, feature_cols)
print(f"Daily dataset shape: {df_daily.shape}")

Creating daily dataset...
Daily dataset shape: (325, 27)


## 6. Data Quality Checks

In [17]:
datasets = {
    '1min': df_imputed,
    '15min': df_15min,
    '30min': df_30min,
    '1h': df_1h,
    '1day': df_daily
}

print("Dataset summary:")
for name, data in datasets.items():
    print(f"\n{name}:")
    print(f"  Shape: {data.shape}")
    print(f"  Date range: {data.index.min()} to {data.index.max()}")
    print(f"  Missing values: {data.isnull().sum().sum()}")
    
    # Check for any infinite values
    inf_count = np.isinf(data.select_dtypes(include=[np.number])).sum().sum()
    print(f"  Infinite values: {inf_count}")

Dataset summary:

1min:
  Shape: (434438, 27)
  Date range: 2023-02-10 00:00:00 to 2023-12-31 14:27:00
  Missing values: 0
  Infinite values: 0

15min:
  Shape: (30925, 27)
  Date range: 2023-02-10 00:00:00 to 2023-12-31 14:15:00
  Missing values: 0
  Infinite values: 0

30min:
  Shape: (15466, 27)
  Date range: 2023-02-10 00:00:00 to 2023-12-31 14:00:00
  Missing values: 0
  Infinite values: 0

1h:
  Shape: (7738, 27)
  Date range: 2023-02-10 00:00:00 to 2023-12-31 14:00:00
  Missing values: 0
  Infinite values: 0

1day:
  Shape: (325, 27)
  Date range: 2023-02-10 00:00:00 to 2023-12-31 00:00:00
  Missing values: 0
  Infinite values: 0


## 7. Save Processed Datasets

In [18]:
# Create data directory if it doesn't exist
data_dir = '../data'
os.makedirs(data_dir, exist_ok=True)

# Save datasets
save_paths = {}

for timeframe, data in datasets.items():
    filename = f'BTCUSD_2023_{timeframe}_cleaned.csv'
    filepath = os.path.join(data_dir, filename)
    
    # Reset index to save timestamp as column
    data_to_save = data.reset_index()
    data_to_save.to_csv(filepath, index=False)
    
    save_paths[timeframe] = filepath
    print(f"Saved {timeframe} dataset to: {filepath}")
    print(f"  File size: {os.path.getsize(filepath) / (1024*1024):.2f} MB")

print("\nAll datasets saved successfully!")

Saved 1min dataset to: ../data\BTCUSD_2023_1min_cleaned.csv
  File size: 177.78 MB
Saved 15min dataset to: ../data\BTCUSD_2023_15min_cleaned.csv
  File size: 12.87 MB
Saved 30min dataset to: ../data\BTCUSD_2023_30min_cleaned.csv
  File size: 6.43 MB
Saved 1h dataset to: ../data\BTCUSD_2023_1h_cleaned.csv
  File size: 3.22 MB
Saved 1day dataset to: ../data\BTCUSD_2023_1day_cleaned.csv
  File size: 0.13 MB

All datasets saved successfully!


## 8. Summary Statistics

In [19]:
print("Processing Summary:")
print(f"Original dataset: {df.shape[0]:,} rows")
print(f"After cleaning: {df_imputed.shape[0]:,} rows")
print(f"Reduction: {df.shape[0] - df_imputed.shape[0]:,} rows removed")

print("\nAggregated datasets:")
for timeframe, data in datasets.items():
    if timeframe != '1min':
        compression_ratio = df_imputed.shape[0] / data.shape[0]
        print(f"{timeframe}: {data.shape[0]:,} rows (compression: {compression_ratio:.1f}x)")

print("\nFeature preservation check:")
print(f"Original features: {len(feature_cols)}")
for timeframe, data in datasets.items():
    preserved_features = len([col for col in feature_cols if col in data.columns])
    print(f"{timeframe}: {preserved_features} features preserved")

Processing Summary:
Original dataset: 484,640 rows
After cleaning: 434,438 rows
Reduction: 50,202 rows removed

Aggregated datasets:
15min: 30,925 rows (compression: 14.0x)
30min: 15,466 rows (compression: 28.1x)
1h: 7,738 rows (compression: 56.1x)
1day: 325 rows (compression: 1336.7x)

Feature preservation check:
Original features: 22
1min: 22 features preserved
15min: 22 features preserved
30min: 22 features preserved
1h: 22 features preserved
1day: 22 features preserved
